In [1]:
from transformers import LlamaForCausalLM, LlamaTokenizer

model = LlamaForCausalLM.from_pretrained("llama-2-7b-hf")
tokenizer = LlamaTokenizer.from_pretrained("llama-2-7b-hf")

Loading checkpoint shards: 100%|██████████| 3/3 [00:02<00:00,  1.31it/s]


In [5]:
model

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (up_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (down_proj): Linear(in_features=11008, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): LlamaRMSNorm()
  )
  (lm_head):

In [10]:
from transformers import pipeline
import torch

device = "cuda:0"

pipeline = pipeline(
    'text-generation',
    model=model,
    torch_dtype=torch.float16,
    device = device,
    tokenizer=tokenizer,
)

sequence = "The temperature in transformers.pipeline means"
output = pipeline(sequence, max_length=512, do_sample=True, temperature=0.9)
print(output)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


[{'generated_text': 'The temperature in transformers.pipeline means that it gets hot, not cold. You don’t want a 1st order effect to be interpreted as a 2nd order effect by thermodynamics, like we are doing here. I could probably prove to you that there is no difference between Φ(t) = Φ(0) + ηt and Φ(t) = Φ(0)eθt in the first two terms, just by adding 1+0t to one side, but you would still insist otherwise, at least until it is pointed out to you that the error in your interpretation is in the third term. You don’t really care about what the temperature is doing, you only care about the 2nd order effect.\nI am no longer posting to your comment section, or even reading your comments on my blog comments. You are not welcome.\n“The “scientific” community is afraid to say that because this would require them to look at and consider the evidence from all the various sources (in the same way that we judge the evidence from a case in a court of law) and not from one cherry-picked result. ”\nTh